<a href="https://colab.research.google.com/github/Jeff-AB/ECE542FinalProject/blob/master/ECE542FinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ECE 542 Final Project
## Upload Rules
Upload your changes to your branch only. When you think that your changes are sufficient, make a pull request to the master branch from your personal branch and assign other members to review it. After a branch has been merged, everyone has to update their setups with the new master branch.

In [0]:
#test